In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

### **Estimación de la población de los municipios a lo largo de los años**

Dado que en el dataset de EGIF hemos introducido el dato de la población actual de los municipios y ha resultado ser relevante para el modelado, hacemos una revisión trando de introducir una estimación más realista de la población en función de los años, puesto que la población ha variado inevitablemente desde 1974.

Para ello se ha ajustado una ecuación lineal a la evolución de la población para cada una de las provincias, para después mediante proporción extrapolarlo a la población de cada uno de los municpios de la provincia correspondiente. A continuación, se detalla el proceso que se ha seguido.

### **Uso del modelo de ecuación lineal de la provincia**

El modelo provincial permite estimar la población de una provincia en cualquier año utilizando la fórmula de una ecuación lineal:

$$
\text{Población Estimada Provincia} = m \cdot (\text{Año}) + b
$$

Donde:
- $m$: Pendiente de la recta que refleja la tasa de cambio anual de la población.
- $b$: Intersección con el eje $y$
- $Año$: Año para el que queremos calcular la población.

### **Estimación de la población del municipio mediante proporcionalidad**
Para estimar la población de un municipio utilizamos los datos del último año de población conocidos y los ajustamos proporcionalmente a la evolución que ha sufrido la población de la provincia suponiendo que la relación entre el municipio y la provincia sea constante a lo largo del tiempo. La fórmula utilizada es la siguiente:

$$
\text{Población estimada municipio} = \text{Población municipio} \cdot \frac{\text{Población estimada provincia}}{\text{Población provincia}}
$$

Donde:
- **Población municipio**: Población actual del municipio
- **Población estimada provincia**: Estimación de la población de la provincia para el año seleccionado.
- **Población provincia**: Problación actual de la provincia.


In [23]:
# Cargar los datos de evolución de la población
poblacion = pd.read_excel('evolucion_poblacion.xlsx')
poblacion.head()

,cod_prov,Provincias,1971,1972,1973,1974,1975,1976,1977,1978,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,1,Álava,199860,205707,211468,217055,223009,228951,235334,241666,...,3.199274e+05,3.198950e+05,3.206634e+05,3.218038e+05,3.238123e+05,3.257392e+05,3.276825e+05,3.298730e+05,3.297577e+05,3.297976e+05
1,2,Albacete,340696,340209,340230,340102,340373,340779,341511,342032,...,3.995097e+05,3.966835e+05,3.949279e+05,3.929575e+05,3.915745e+05,3.903371e+05,3.895279e+05,3.898739e+05,3.888801e+05,3.877595e+05
2,3,Alicante/Alacant,922357,944318,967402,989556,1013107,1037097,1062433,1087208,...,1.854244e+06,1.850624e+06,1.842174e+06,1.842446e+06,1.842572e+06,1.845470e+06,1.862780e+06,1.885559e+06,1.895192e+06,1.904315e+06
3,4,Almería,377528,380655,383898,386986,390733,394591,398954,403095,...,6.916799e+05,6.888880e+05,6.911546e+05,6.961858e+05,7.000410e+05,7.010689e+05,7.068710e+05,7.156654e+05,7.208610e+05,7.240312e+05
4,5,Asturias,1051557,1059352,1067050,1073592,1082341,1091460,1102051,1112156,...,1.694579e+05,1.673314e+05,1.657408e+05,1.636503e+05,1.617487e+05,1.600324e+05,1.592596e+05,1.588954e+05,1.588541e+05,1.590369e+05


In [24]:
# Cargamos los datos del dataset del egif
bdif = pd.read_excel('bdif_geo_clim_fwi.xlsx')
bdif.head()

,parte,año,cod_com,cod_prov,probignicion,diastormenta,diasultimalluvia,tempmaxima,humrelativa,velocidadviento,...,lat,indicativo,prec,tmax,dir,velmedia,racha,sol,hrmedia,fwi
0,1974020249,1974,11,2,0.0,NaN,NaN,10.0,75.0,30.0,...,38.366800,8175,0.0,17.0,29.0,6.1,11.9,9.5,56.0,12.101361
1,1974020374,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,38.512190,8175,0.0,26.6,18.0,6.4,13.1,10.9,53.0,69.039001
2,1974020459,1974,11,46,0.0,NaN,NaN,NaN,NaN,NaN,...,39.059798,8414A,0.0,23.0,9.0,3.6,11.1,3.3,71.0,11.837668
3,1974022274,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,38.499810,8175,0.0,28.0,NaN,5.0,NaN,8.9,63.0,42.358257
4,1974022457,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,38.552177,8175,0.0,23.2,NaN,4.7,NaN,9.3,45.0,15.454229


In [25]:
# Hay que trasponer los años y la población 
poblacion = poblacion.melt(id_vars=['cod_prov', 'Provincias'], var_name='Año', value_name='Población')

# Pasamos año a númerico
poblacion['Año'] = poblacion['Año'].astype(int)

# Mostramos
poblacion.head()

,cod_prov,Provincias,Año,Población
0,1,Álava,1971,199860.0
1,2,Albacete,1971,340696.0
2,3,Alicante/Alacant,1971,922357.0
3,4,Almería,1971,377528.0
4,5,Asturias,1971,1051557.0


In [27]:
# Definimos la ecuación lineal con la que modelamos la evolución de la población
def ecuacion_lineal(x, m, b):
    return m * x + b

# Creamos un diccionario vacío que guarde la ecuación de evolución de la población para cada provincia
ecuacion_provincia = {}

for cod in poblacion['cod_prov'].unique():
    # Recuperamos los datos de población para cada año de la provincia
    datos = poblacion[poblacion['cod_prov'] == cod]
    fechas = datos['Año'].values
    poblaciones = datos['Población'].values

    # Obtenemos la ecuación lineal y guardarmos los parámetros, obviamos la varianza
    parametros, _ = curve_fit(ecuacion_lineal, fechas, poblaciones)
    ecuacion_provincia[cod] = parametros

# Mostramos el resulado
print(ecuacion_provincia)

{1: array([ 2.25772797e+03, -4.22696977e+06]), 2: array([ 1.40757697e+03, -2.44721538e+06]), 3: array([ 2.07560562e+04, -4.00031832e+07]), 4: array([ 7.80825259e+03, -1.50584539e+07]), 5: array([-2.63000934e+04,  5.32109107e+07]), 6: array([ 1.36275341e+04, -2.68160452e+07]), 7: array([ 1.25296533e+04, -2.41838535e+07]), 8: array([ 1.36376314e+05, -2.69618972e+08]), 9: array([-1.12800200e+05,  2.27965804e+08]), 10: array([ 1.10712938e+03, -1.83314975e+06]), 11: array([ 2.23507205e+04, -4.38644883e+07]), 12: array([-1.10487826e+04,  2.28850349e+07]), 13: array([1.44114756e+02, 2.21971448e+05]), 14: array([ 1.04370641e+04, -2.02545019e+07]), 15: array([ 1.78668124e+04, -3.49173894e+07]), 16: array([-1.48331310e+04,  3.01300082e+07]), 17: array([-9.55089425e+03,  1.99939317e+07]), 18: array([ 1.91834561e+04, -3.77953901e+07]), 19: array([-5.76542449e+03,  1.18886719e+07]), 20: array([-1.49422236e+03,  3.72793355e+06]), 21: array([ 1.04002402e+04, -2.04635579e+07]), 22: array([-1.26139819e

In [30]:
# Población base de las provincias la obtenemos del dataset de población para el último año disponible
poblacion_base_provincia = poblacion[poblacion['Año'] == 2022].set_index('cod_prov')['Población'].to_dict()

# Creamos una función que haga una estimación de la población del municipio en función del que se hace en la provincia
def estimar_poblacion(fila, ecuacion_provincia, poblacion_provincia):
    """
    Calcula la población estimada de un municipio basado en su proporción con la población de la provincia y la estimación
    de la población de la provincia.

    Parametros:
    fila: fila del dataframe de donde extraemos el año, el codigo de la provincia y la poblacion del municipio
    ecuacion_provincia: diccionario que contiene las ecuaciones lineales de evolución de la población por provincia
    poblacion_provincia: diccionario que contiene las poblaciones actuales de las provincias
    
    Retorna:
    float: Población estimada para el municipio.
    """
    cod_prov = fila['cod_prov']
    ano = fila['año']
    poblacion_base_mun = fila['poblacion']
    
    if cod_prov in ecuacion_provincia:
        # Obtenemos la ecuación lineal de la provincia
        m, b = ecuacion_provincia[cod_prov]
        
        # Calculamos la población estimada en el año correspondiente
        poblacion_est_prov = m * ano + b
        
        # Obtener población base de la provincia
        poblacion_base_prov = poblacion_provincia.get(cod_prov)
        
        # Calcular población estimada del municipio por proporcionalidad con la de la provincia
        poblacion_estimada = poblacion_base_mun * (poblacion_est_prov / poblacion_base_prov)
        
        # Si la población estimada es negativa, asignar 1
        if poblacion_estimada < 0:
            poblacion_estimada = 1
        
        return poblacion_estimada

# Aplicar la función al dataset bdif
bdif['poblacion_estimada'] = bdif.apply(estimar_poblacion, axis=1, args=(ecuacion_provincia, poblacion_base_provincia))

# Mostramos los resultados
bdif.head()

,parte,año,cod_com,cod_prov,probignicion,diastormenta,diasultimalluvia,tempmaxima,humrelativa,velocidadviento,...,indicativo,prec,tmax,dir,velmedia,racha,sol,hrmedia,fwi,poblacion_estimada
0,1974020249,1974,11,2,0.0,NaN,NaN,10.0,75.0,30.0,...,8175,0.0,17.0,29.0,6.1,11.9,9.5,56.0,12.101361,2117.457786
1,1974020374,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,8175,0.0,26.6,18.0,6.4,13.1,10.9,53.0,69.039001,26076.005568
2,1974020459,1974,11,46,0.0,NaN,NaN,NaN,NaN,NaN,...,8414A,0.0,23.0,9.0,3.6,11.1,3.3,71.0,11.837668,3790.498405
3,1974022274,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,8175,0.0,28.0,NaN,5.0,NaN,8.9,63.0,42.358257,1133.070631
4,1974022457,1974,11,2,0.0,NaN,NaN,NaN,NaN,NaN,...,8175,0.0,23.2,NaN,4.7,NaN,9.3,45.0,15.454229,513.556146


In [31]:
bdif['poblacion_estimada'].describe()

count    5.885540e+05
mean     1.387063e+04
std      6.339124e+04
min      1.000000e+00
25%      1.061466e+03
50%      2.688832e+03
75%      8.146365e+03
max      3.034878e+06
Name: poblacion_estimada, dtype: float64

In [32]:
bdif.to_excel("bdif_geo_clim_fwi_pob.xlsx", index=False)